In [1]:
mkdir -p deseq

cp ../../R/no_dup/results/*_deseq/*res_2_0.01.tsv deseq/.
cp ../../R/no_dup/data_info.tsv .

In [2]:
sed -i 's/"//g' data_info.tsv

In [3]:
dos2unix data_info.tsv
dos2unix deseq/*

In [4]:
ls deseq/* | while read file; do
    name=$(basename -s '.tsv' $file)
    exp=$(echo "$name" | sed 's/_deseq_res_2_0.01//g' | cut -d'_' -f1)
    ref=$(echo "$name" | sed 's/_deseq_res_2_0.01//g' | cut -d'_' -f3)
    
    n=$(wc -l $file | awk '{print $1}')
    exp_col=$(awk -v str="$exp" -v n="$n" 'BEGIN { for (i=1; i<n; i++) print str }')
    ref_col=$(awk -v str="$ref" -v n="$n" 'BEGIN { for (i=1; i<n; i++) print str }')
    
    mo=$(cat $file | sed '1d' | cut -f1)
    deg=$(cat $file | sed '1d' | cut -f2-)
    
    paste <(echo "$mo") <(echo "$exp_col") <(echo "$ref_col") <(echo "$deg")
done > genes.tmp

In [ ]:
cat genes.tmp | while read line; do
    lt=$(echo "$line" | cut -f4)
    lt_info=$(grep "$lt" data_info.tsv | cut -f3-)
    echo "$lt_info"
done | paste <(cat genes.tmp) - | sed '1i species\texperiment\treference\tlocus_tag\tlog2FC\tp_adjusted\tregulation\tproduct\tgene' > genes_pre.tmp

In [ ]:
grep -v "rRNA\|ribosomal protein" genes_pre.tmp > genes.tsv
rm *.tmp